In [ ]:
# 使用snap, debian buster下載chromium
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [1]:
#連結雲端硬碟
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks/stock_crawler")

Mounted at /content/drive


In [4]:
import os
if "EPS_files" not in os.listdir():
  os.mkdir("/content/drive/MyDrive/Colab Notebooks/stock_crawler/EPS_files")
if "EPS_files" not in os.listdir():
  os.mkdir("/content/drive/MyDrive/Colab Notebooks/stock_crawler/EPS_figs")
os.listdir()

['stock_crawer.ipynb',
 'stock_list.txt',
 '.ipynb_checkpoints',
 'EPS_files',
 'time spended per request to thread number.png',
 'EPS_figs',
 'crawer.ipynb',
 'geckodriver.log',
 'chromedriver.exe']

In [ ]:
#爬蟲程式定義
import time
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException
from concurrent.futures.thread import ThreadPoolExecutor

finished = set(map(lambda x: x[:-8], os.listdir("EPS_files")))
# get N * 50 season data
N = 5
load_time = 0.3
# how many thread to create
Thread_number = 4

chrome_options = webdriver.ChromeOptions()
# 添加 User-Agent
chrome_options.add_argument(
    'user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"')
# 指定瀏覽器解析度
chrome_options.add_argument('window-size=1920x1080')
# 不載入圖片，提升速度
chrome_options.add_argument('blink-settings=imagesEnabled=false')
# 瀏覽器不提供視覺化頁面
chrome_options.add_argument('--headless')
# 最高權限
chrome_options.add_argument('--no-sandbox')
# 改寫docker記憶體位置
chrome_options.add_argument('--disable-dev-shm-usage')

drivers = [webdriver.Chrome("/usr/bin/chromedriver", chrome_options=chrome_options) for _ in range(Thread_number)]

def keybroad_action(browser, key, repeat=1, sleep=0):

    for _ in range(repeat):
        actions = ActionChains(browser)
        actions.send_keys(key)
        actions.perform()
        time.sleep(sleep)
    return

def crawing(stock, browser):
    if stock in finished:
      print(f"{stock} already collected")
      return
    print(f"Ready to get: {stock}")
    try:
        browser.get(f"https://invest.cnyes.com/usstock/detail/{stock}/earnings/estimate#fixed")
        
        time.sleep(3)
        keybroad_action(browser, Keys.TAB, 19)
        keybroad_action(browser, Keys.ENTER, N - 1, load_time)
    except Exception as e:
        print(e)
        print(f"not found stock ticker {stock}")
        return
    try:
        with open(f"EPS_files/{stock}_EPS.txt", "w") as f:
            page_results = browser.find_elements("xpath", "//tbody")
            f.writelines(item.text for item in page_results)
            keybroad_action(browser, Keys.TAB)
            for i in range(1, N):
                WebDriverWait(browser, 1).until(
                    EC.visibility_of_element_located((By.XPATH, f'//*[@id="DataTables_Table_0_paginate"]/span/a[{i}]')))
                time.sleep(load_time)
                keybroad_action(browser, Keys.TAB)
                keybroad_action(browser, Keys.ENTER)
                page_results = browser.find_elements("xpath", "//tbody")
                f.writelines(item.text for item in page_results)
        print(f"finish to get: {stock}")
    except TimeoutException:
        print(f"got {i} page on {stock}")
    except IndexError:
        print(f"{stock} IndexError on step 2")
    except:
        print(f"other exception on {stock} on step 2")

ModuleNotFoundError: ignored

In [ ]:
#爬蟲程式執行
with open("stock_list.txt", "r") as f:
    stock_list = f.read().split()

for i in range(0, len(stock_list), Thread_number):
  with ThreadPoolExecutor(max_workers=Thread_number) as executor:
    bucket = executor.map(crawing, stock_list[i:i+Thread_number], drivers)

Ready to get: ME
Ready to get: VXI
Ready to get: DIBSCOE already collected

VXI may not complete
ME may not complete
DIBS may not complete
Ready to get: YI
Ready to get: LAX
JFU already collected
Ready to get: NMTR
LAX may not complete
NMTR may not complete
YI may not complete
Ready to get: MASS
Ready to get: SCWO
Ready to get: FEAM
Ready to get: VCXB
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
not found stock ticker VCXB
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
not found stock ticker FEAM
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
not found stock ticker MASS
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
not found stock ticker SCWO


KeyboardInterrupt: ignored

In [ ]:
#檢查已爬數量
import os
print(len(os.listdir("EPS_files")))

772


In [ ]:
#畫圖
import re
from matplotlib import pyplot as plt
import os

finished = set(map(lambda x: x[:-14], os.listdir("EPS_figs")))

def drawACC4season(parsed, stock):
    times = [p[4:8] for p in parsed]
    x = [i for i in range(3, len(parsed))]
    EPSs = [float(p[-5:]) for p in parsed]
    acc_ESPs = []
    tmp = sum(EPSs[:3])
    for i in range(3, len(EPSs)):
        tmp += EPSs[i]
        acc_ESPs.append(tmp)
        tmp -= EPSs[i - 3]
    plt.bar(x, acc_ESPs)
    plt.title(stock)
    plt.ylabel("accumulate 4 season ESP")
    plt.xlabel("time")
    plt.xticks(x, times[3:], rotation=90)
    ax = plt.gca()
    ax.xaxis.set_major_locator(plt.MultipleLocator(4))
    plt.savefig(f"EPS_figs/{stock}_EPS_image.jpg")
    plt.cla()

def parsing(stock):
    if stock in finished:
      return
    with open(f"EPS_files/{stock}_EPS.txt", "r") as f:
        data = f.read()
    parsed = re.findall(r"\(Q\d\s\d{4}\)\s-?\d\.\d\d", data)
    if len(parsed) == 0:
        return None, None
    parsed_set = set(parsed)
    if len(parsed) == len(parsed_set):
        parsed.reverse()
    else:
        parsed = sorted(parsed_set, key=lambda x: (int(x[4:8]), int(x[2])))
    return (parsed, stock)
for f in os.listdir("EPS_files"):
    parsed, stock = parsing(f[:-8])
    if parsed is not None:
        drawACC4season(parsed, stock)
drive.flush_and_unmount()

In: VXI
removed: VXI
In: ME
removed: ME
In: DIBS
removed: DIBS


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-c45a781281b6>", line 49, in <module>
    drive.flush_and_unmount()
  File "/usr/local/lib/python3.8/dist-packages/google/colab/drive.py", line 92, in flush_and_unmount
    out, err = p.communicate()
  File "/usr/lib/python3.8/subprocess.py", line 1028, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/lib/python3.8/subprocess.py", line 1868, in _communicate
    ready = selector.select(timeout)
  File "/usr/lib/python3.8/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceba

KeyboardInterrupt: ignored